# FCC Auction Bid Optimization

The FCC often auctions off licenses to use radio spectrum in frequency chunks that are called blocks. They have mapped out 416 geographical regions call Partial Economic Areas (PEA), each of which have a set number of licenses available in the auction.  For this example, there are 411 PEAs and each have 24 licenses available.  The licenses are named M1 through M10 and N1 through N14 and their frequencies are all contiguous in that order. The data file provided lists each PEA and it's population per MHz of spectrum (MHzPOPs) for each license.  The greater the MHzPOPs the more potential customers that can be served by a license in that PEA.  The ultimate goal of participating in this auction is to maximize the MHzPOPs of the licenses won in the auction.  The data file also lists the price for each license, which varies significantly between licenses.  You must decide which licenses to buy with some restrictions.  First, you have a total budget of $100 million to spend. Next, you want to win at most 4 licenses in any single PEA but require at least 2 licenses for the spectrum to be useful to you.  Finally, you can only buy licenses that are contiguous (i.e. next to each other). 

## (COLAB ONLY) Install OR-tools

In [1]:
!pip install ortools

     |████████████████████████████████| 14.5 MB 6.7 MB/s 
     |████████████████████████████████| 126 kB 37.9 MB/s 
     |████████████████████████████████| 1.1 MB 36.5 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.12.0
    Uninstalling absl-py-0.12.0:
      Successfully uninstalled absl-py-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.4.0 requires absl-py<0.13,>=0.9, but you have absl-py 1.0.0 which is incompatible.


## Import our 3rd party libraries

In [4]:
from ortools.linear_solver import pywraplp
import pandas as pd

## Import our data file using pandas

Remember, the data file has some valuable information in it.  It lists the maximum licenses we would like to win in each PEA (happens to be constant at 4).  It also lists the MHzPOPs for each PEA which is necessary for our objective function.  Finally, it also has the price of each license.

In [57]:
data = pd.read_csv('values.csv')

## Create our solver

Even though we have not yet talked about the linear programming model we are building we know that it is going to be an Integer Programming problem because we are selecting some subset of fixed things (licenses).  Therefore we create an or-tools solver using SCIP which is an integer programming library.

In [58]:
solver = pywraplp.Solver.CreateSolver('SCIP')

## Define Variables

Now we need to start defining our model.  I always recommend doing this by hand first on paper or a whiteboard.  After you have it all mapped out, then you can program it.  The way you define it in or-tools is very very similar to how you write it out by hand.  

Since our goal is to choose which licenses are optimal to select from some group of available licenses, we would naturally think to use binary variables to represent if a license is optimal to select.  However, looking a little further down the line we know that if we choose multiple licenses in a PEA they must all be contiguous.  This may be possible with these variables and some smart constrains but I propose a different solution.  To me, it make more sense to create a variable that represents a number of contiguous licenses.  Here is what the licenses for a single PEA look like:

|M1|M2|...|M10|N1|N2|...|N14|
| --- | --- | --- | --- | --- | --- | --- | --- |

If you want 3 licenses, you would have (24 - 3 + 1 =) 22 variables that would represent the following assignments:

|License 1| License 2| License 3|
| --- | --- | --- |
|M1|M2|M3|
|M2|M3|M4|
|M3|M4|M5|
|...|...|...|
|N12|N13|N14|

However, since our demand is not fixed, we would also need to create these variables for the possibility of 2 and 4 license combinations.  Therefore, we want to iterate through each PEA and create a variable that represents 4, 3, and 2 contiguous licenses along with storing the cost and MHzPOPs for that assignment.

In [59]:
cost = {}
mhz_pop = {}
assignments = {}

for row in range(len(data)):
  for blocks in range(2, data['Demand'][row] + 1):
    for index in range(24 - blocks + 1):
      key = (data['PEA'][row],blocks,index)
      assignments[key] = solver.IntVar(0,1,str(key))
      # since our data has 5 columns before the license prices start we need to shift our indexing by 5
      cost[key] = sum(data.iloc[row][(5 + index):(5 + index + blocks)])
      mhz_pop[key] = data['MHzPOPs'][row] * blocks

## Create objective function

Now that we have created our variables, we can define our objective function.  To reiterate, our objective is to maximize MHzPOPs.

In [60]:
solver.Maximize(sum([mhz_pop[key] * key[1] * assignments[key] for key in assignments.keys()]))

## Add constraints

Our only real constraints here is our budget and that we do not want to win more than one assignment per PEA. 

### Budget Constraint

In [64]:
solver.Add(sum([cost[(pea, blocks, index)] * assignments[(pea, blocks, index)] for pea in data['PEA'] for blocks in range(2,5) for index in range(24 - blocks + 1)]) <= 100000000)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x0000026E0BD3DC30> >

### Single assignment constraints

Notice this is a loop that creates a constraint per PEA, so it is multiple constraints.  Each of these constraints says that we want at most 1 assignment per PEA.

In [62]:
for pea in data['PEA']:
    solver.Add(sum([assignments[(pea, blocks, index)] for blocks in range(2,5) for index in range(24 - blocks + 1)]) <= 1)

## Solve

Now all we need to do is solve and check that the result is optimal.  If it is we should print our results.

In [65]:
status = solver.Solve()

licenses = ['M' + str(i + 1) for i in range(10)] + ['N' + str(i + 1) for i in range(14)]

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Total MHzPOPs =', solver.Objective().Value())
    print('Licenses Won:')
    spending = 0
    for pea in data['PEA']:
        for blocks in range(2,5):
            for index in range(24 - blocks + 1):
                key = (pea, blocks, index)
                if assignments[key].solution_value() == 1:
                    spending += cost[(pea, blocks, index)]
                    message = str(key) + " " + pea + ': won ' + str(blocks) + ' licenses ' + str(licenses[index:(index + blocks)])
                    print(message)
    print(f'Total Spent = {spending}')
else:
    print('The problem does not have an optimal solution.')

Solution:
Total MHzPOPs = 375775751700.0
Licenses Won:
('PEA001', 2, 4) PEA001: won 2 licenses ['M5', 'M6']
('PEA004', 4, 14) PEA004: won 4 licenses ['N5', 'N6', 'N7', 'N8']
('PEA005', 4, 1) PEA005: won 4 licenses ['M2', 'M3', 'M4', 'M5']
('PEA006', 4, 19) PEA006: won 4 licenses ['N10', 'N11', 'N12', 'N13']
('PEA007', 2, 17) PEA007: won 2 licenses ['N8', 'N9']
('PEA008', 4, 10) PEA008: won 4 licenses ['N1', 'N2', 'N3', 'N4']
('PEA009', 4, 20) PEA009: won 4 licenses ['N11', 'N12', 'N13', 'N14']
('PEA011', 3, 4) PEA011: won 3 licenses ['M5', 'M6', 'M7']
('PEA012', 4, 1) PEA012: won 4 licenses ['M2', 'M3', 'M4', 'M5']
('PEA013', 4, 0) PEA013: won 4 licenses ['M1', 'M2', 'M3', 'M4']
('PEA014', 4, 7) PEA014: won 4 licenses ['M8', 'M9', 'M10', 'N1']
('PEA016', 4, 12) PEA016: won 4 licenses ['N3', 'N4', 'N5', 'N6']
('PEA017', 4, 10) PEA017: won 4 licenses ['N1', 'N2', 'N3', 'N4']
('PEA018', 4, 4) PEA018: won 4 licenses ['M5', 'M6', 'M7', 'M8']
('PEA019', 4, 16) PEA019: won 4 licenses ['N7', '